In [ ]:
import pandas as pd

from constants.abs_paths import AbsDirPath


df = pd.read_csv(AbsDirPath.ROOT / "processing_pipeline" / "analysis" / "param_tuning.csv")

## Processing metrics per items

In [ ]:
from processing_pipeline.utilities.data_transformation import load_all_files

df_items = load_all_files(AbsDirPath.PARAMETER_TUNING_DIR)
df_items.shape

In [ ]:
df_items.head()

In [ ]:
n_items = df_items.shape[0]
df["time_per_item"] = df.time / n_items
df.head()

In [ ]:
min_value = df.time_per_item.min()
df["time_per_item_delta"] = df.time_per_item - min_value
df["time_per_item_delta"] = df.time_per_item_delta.astype("float").round(5)
df["time_per_item_delta"] = df.time_per_item_delta.astype("str").apply(lambda x: x if x == "0" else "+" + x)
df.head()

In [ ]:
items_filter_rate_per_stage = .75
est_nitems_s0 = 524909
est_nitems_s1 = 1241367 * items_filter_rate_per_stage
est_nitems_s2 = 524909 * (items_filter_rate_per_stage ** 2)
est_nitems_s3 = 524909 * (items_filter_rate_per_stage ** 3)

est_nitems_s0, est_nitems_s1, est_nitems_s2, est_nitems_s3

In [ ]:
seconds_per_day = 24 * 60 * 60
df["est_time_s0_days"] = est_nitems_s0 * df.time_per_item / seconds_per_day
df["est_time_s1_days"] = est_nitems_s1 * df.time_per_item / seconds_per_day
df["est_time_s2_days"] = est_nitems_s2 * df.time_per_item / seconds_per_day
df["est_time_s3_days"] = est_nitems_s3 * df.time_per_item / seconds_per_day

df["est_time_s0_days"] = df.est_time_s0_days.astype("float").round(2)
df["est_time_s1_days"] = df.est_time_s1_days.astype("float").round(2)
df["est_time_s2_days"] = df.est_time_s2_days.astype("float").round(2)
df["est_time_s3_days"] = df.est_time_s3_days.astype("float").round(2)

df["est_time_total_days"] = df[["est_time_s0_days", "est_time_s1_days", "est_time_s2_days", "est_time_s3_days"]].sum(axis=1)
two_server_boost = .75
df["est_time_total_2machines_days"] = df.est_time_total_days * two_server_boost

## Analyze results

In [ ]:
df.head()

In [ ]:
df.sort_values("time")

In [ ]:
df.pivot_table(index=["model_name"], columns=["n_threads" , "n_batches"], values="time", aggfunc="mean")

In [ ]:
df.pivot_table(index=["model_name"], columns=["n_batches"], values="time", aggfunc="mean")

In [ ]:
df.pivot_table(index=["model_name"], columns=["n_threads"], values="time", aggfunc="mean")

In [ ]:
df.groupby(["model_name", "n_threads", "n_batches"]).time.agg("mean")

In [ ]:
df.loc[df.groupby(["model_name", "n_threads"]).time.idxmin()].groupby(["model_name", "n_threads", "n_batches"]).time.agg("mean")

In [ ]:
df.loc[df.groupby(["model_name"]).time.idxmin()]